# TBL: Orca image normalization workflow

This notebook shows how to use the workflow to compute normalized images recorded by the Orca detector on the TBL instrument.

In [ ]:
from ess.tbl import orca
from ess.imaging.types import *
import scipp as sc
import plopp as pp

%matplotlib widget

## Workflow setup

In [ ]:
wf = orca.OrcaNormalizedImagesWorkflow()

wf[Filename[SampleRun]] = "ymir_lego_sample_run.hdf"
wf[Filename[DarkBackgroundRun]] = "ymir_lego_dark_run.hdf"
wf[Filename[OpenBeamRun]] = "ymir_lego_openbeam_run.hdf"
wf[NeXusDetectorName] = 'orca_detector'

wf[MaskingRules] = {}  # No masks to begin with
wf[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.upper_bound

In [ ]:
wf.visualize(NormalizedImage, compact=True, graph_attr={"rankdir": "LR"})

## Run the workflow

We compute the final normalized image:

In [ ]:
image = wf.compute(NormalizedImage)
image

In [ ]:
pp.slicer(image)

## Adding masks

If we want to mask some part of the image, we update the masking rules.
For example, here we mask the upper part of the image:

In [ ]:
wf[MaskingRules] = {'y_pixel_offset': lambda x: x > sc.scalar(0.082, unit='m')}

pp.slicer(wf.compute(NormalizedImage))

## Intermediate results

We can also inspect intermediate results, which is useful for debugging:

In [ ]:
results = wf.compute([
    RawDetector[SampleRun],
    CorrectedDetector[SampleRun],
    BackgroundSubtractedDetector[SampleRun]
])

fig = pp.tiled(2, 2, hspace=0.3, wspace=0.3)
fig[0, 0] = results[RawDetector[SampleRun]]['time', 0].plot(title='Raw data')
fig[0, 1] = results[CorrectedDetector[SampleRun]]['time', 0].plot(title='Masks applied')
fig[1, 0] = results[BackgroundSubtractedDetector[SampleRun]]['time', 0].plot(title='Background subtracted')
fig[1, 1] = image['time', 0].plot(title='Final image')
fig